In [1]:
pip install snscrape

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install yfinance

  Using cached yfinance-0.1.55.tar.gz (23 kB)
  Using cached multitasking-0.0.9.tar.gz (8.1 kB)
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=d7f772c5e731e9fef79d90c0ffc4cc71af357cd8e33795fc52324d55458a5f16
  Stored in directory: /Users/dsw46/Library/Caches/pip/wheels/aa/8a/36/59ed4f6fbcb6100967618eeb0696046bf9777a41ac2ff1f9b9
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8367 sha256=722989bb5ab621ff7a432e92730fde5f5ca892f6ddec29754035ee101fc68bab
  Stored in directory: /Users/dsw46/Library/Caches/pip/wheels/ae/25/47/4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import itertools
import snscrape.modules.twitter as sntwitter
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
import re
import yfinance as yf
import datetime

In [46]:
df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('from: MadsC007').get_items(), 100))

In [37]:
a = 'MadsC007'

In [38]:
df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('from: '+a).get_items(),1000))

In [41]:
df['date'][0]

Timestamp('2021-01-24 10:10:27+0000', tz='UTC')

In [91]:
test= df['content'][14]

In [92]:
test

'This illustrates the power of being an aggretor of demand and suply: \n\nShopee is making a fintech company like $APT.AX, Klarna and $AFRM without feeling the need to make updates to the market. $PYPL from EBay and MercadoPago on $MELI platform are simular examples https://t.co/xZDWztRF2x'

In [26]:
#Should only return a potential ticker if there's 2, 3, or 4 capital letters preceeded by
# a '$' and not followed by a capital letter.
potential_ticker = []
for i in range(len(test)-3):
    if test[i] == '$':
        if test[i+1].isalpha():
            if test[i+2].isalpha():
                if not test[i+3].isalpha():
                    potential_ticker.append(test[i+1]+test[i+2])
                if test[i+3].isalpha():
                    if not test[i+4].isalpha():
                        potential_ticker.append(test[i+1]+test[i+2]+test[i+3])
                    if test[i+4].isalpha():
                            potential_ticker.append(test[i+1]+test[i+2]+test[i+3]+test[i+4])

NameError: name 'test' is not defined

In [60]:
potential_ticker

['APT', 'AFRM', 'PYPL', 'MELI']

In [48]:
ticker_ESG('TSLA')

31

# Building an all-cointaining function which takes a twitter handle as input and outputs info about the companies this person mostly tweets about

Dependent functions:

In [68]:
def tickers_in_tweet(tweet):
    """ Input: A single tweet as a string
        
        Output: A list of all the tickers mentioned
        
        Will return a ticker if there's 2, 3, or 4 capital 
        letters preceeded by a '$' and not followed by a capital letter.      
        
    """


    tickers = []
    for i in range(len(tweet)-4):
        if tweet[i] == '$':
            if tweet[i+1].isalpha():
                if tweet[i+2].isalpha():
                    if not tweet[i+3].isalpha():
                        tickers.append(tweet[i+1]+tweet[i+2])
                    if tweet[i+3].isalpha():
                        if not tweet[i+4].isalpha():
                            tickers.append(tweet[i+1]+tweet[i+2]+tweet[i+3])
                        if tweet[i+4].isalpha():
                                tickers.append(tweet[i+1]+tweet[i+2]+tweet[i+3]+tweet[i+4])
    return tickers

#Web scraping for ESG

def ticker_ESG(ticker):
    """This function accepts a ticker input and outputs its Total ESG Risk Score"""
    #Generate URL
    working_url = 'https://uk.finance.yahoo.com/quote/'+str(ticker).upper()+'/sustainability?p='+str(ticker).upper()
    total_ESG = scrape_yahoo(working_url)
    return total_ESG

def scrape_yahoo(url):
    """This function checks and accepts a Yahoo finance URL and outputs the Total ESG Risk Score as an int """

    #GET page and convert content to bs4 soup
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')


    #Find class containing ESG risk score from 0-100, cast to string
    total_ESG = str(soup.findAll('div', {"class": "Fz(36px) Fw(600) D(ib) Mend(5px)"}))

    #Extract ESG score using regex, with error handling
    if total_ESG != '[]':
        total_ESG = int(re.search('data-reactid="20">(.+?)</div>', total_ESG).group(1))
    else:
        total_ESG = None

    return total_ESG

#Testing
#print(ticker_ESG('STT'))


#Calculate average ESG of all companies mentioned

def ESG_avg_function(dictionary):
    """
    Input: dictionary with tickers as keys and number of mentions as values
    Output: average ESG score, weighted according to the number of mentions
    """
    #Numerator = sum(ESG-score * number of mentions)
    #The sum is over all companies which have an ESG score\
    
    numerator = 0
    total_mentions = 0 
    
    keys_list = list(dictionary)
    
    for i in range(len(keys_list)):
        ESG_score = ticker_ESG(keys_list[i])
        if not ESG_score == None:
            numerator += ESG_score * dictionary[keys_list[i]]
            total_mentions += dictionary[keys_list[i]]
        
    ESG_avg = numerator/total_mentions
    
    return ESG_avg
    

#Stock price functions
def stock_price_today(ticker):
    """
    Input: ticker of a stock (as string)
    Returns the stock price of that stock today
    """
    now = datetime.datetime.now().strftime("%Y-%m-%d")
    stock = yf.Ticker(ticker)
    price = data.history(start='2021-01-01',  end=now)
    price_today = price['Close'][-1]
    
    return price_today

def stock_price_at_date(ticker, date):
    """
    Inputs: 1) ticker of a stock (as string)
            2) Date you want the stock price for (format: 'YYYY-MM-DD')
    Returns the stock price of that ticker for that day (or closest to)
    """
    now = datetime.datetime.now().strftime("%Y-%m-%d")
    stock = yf.Ticker(ticker)
    price = data.history(start=date,  end=now)
    price_date = price['Close'][0]
    
    return price_date


def stock_return_since_mention(ticker, date):
    """
    Input:  Ticker
            Date the stock was mentioned (format: 'YYYY-MM-DD')
    Output: Stock return since it was mentioned
    """
    price_today = stock_price_today(ticker)
    price_date = stock_price_at_date(ticker, date)
    
    stock_return = (price_today-price_date)/price_date
    return stock_return
    

Main function:

In [81]:
def twitter_scrape(handle, num_tweet):
    """
    Input:  1) handle: Twitter handle of a person without '@' (e.g. JoeBiden)
            2) num_tweet: The number of past tweets the analysis will be based on (e.g. 100)
    
    Outputs: 1) most_tweeted_company: The company this person has tweeted most times
             2) ESG_most_tweeted_company: The ESG score of the company mentioned above
             3) avg_esg: The average ESG score by all company this person has mentioned 
               (weighted by number of mentions)
    """
    df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('from: '+handle).get_items(), num_tweet))
    date_df = df['date'].dt.date
    sum_stock_return = 0
    
    #gathers list of all tickers mentioned in all tweets
    tickers_list = []
    for i in range(len(df)):
        new_tickers = []
        new_tickers = tickers_in_tweet(df['content'][i])
        tickers_list += new_tickers
        #If a tweet is added to the list calculate stock returns for this batch of stocks
        if new_tickers != []:
            date = str(date_df[i])
            for j in range(len(new_tickers)):
                ticker = new_tickers[j]
                stock_return = stock_return_since_mention(ticker, date) 
                sum_stock_return += stock_return
    
    #Average stock return = sum(stock_return)/number of stock mentions
    number_of_stock_mentions = len(tickers_list)
    average_stock_return = sum_stock_return/number_of_stock_mentions
    
    #Create dict containing all companies mentioned (key), and the frequency (value)
    ticker_dict = {}
    for i in range(len(tickers_list)):
        try: 
            ticker_dict[tickers_list[i]]+=1
        except KeyError:
            ticker_dict[tickers_list[i]] = 1
    
    max_key = max(ticker_dict, key=ticker_dict.get)
    most_tweeted_company = max_key
    number_of_mentions = ticker_dict[most_tweeted_company]
    
    ESG_most_tweeted_company = ticker_ESG(most_tweeted_company)
    
    try:
        ESG_avg = ESG_avg_function(ticker_dict)
    except ZeroDivisionError:
        ESG_avg = 'This person has not tweeted about any ESG-rated companies!'
    
    return most_tweeted_company, number_of_mentions, ESG_most_tweeted_company,\
            ESG_avg, average_stock_return
    
    
    
    
    
    
    

In [83]:
twitter_scrape('WarlusTrades',50)

('PLTR', 2, None, 21.333333333333332, 0.05445628339957298)

In [ ]:
#WarlusTrades
#MadsC007

#### Use yfinance package to get both historic and current stock price, to calculate return since it was mentioned

In [9]:
msft = yf.Ticker("MSFT")

# get stock info
#print(msft.info)

# get historical market data
#hist = msft.history(period="5d")

In [66]:
stock_return_since_mention('aapl',str(new_df[0]))

- AAPL: No data found for this date range, symbol may be delisted


IndexError: index 0 is out of bounds for axis 0 with size 0

In [55]:
df['date'][0]

Timestamp('2021-01-24 10:10:27+0000', tz='UTC')

'2021-01-24'

In [78]:
tickers_in_tweet('hello') != []

False

In [79]:
['a', 'b'] + ['c', 'd']

['a', 'b', 'c', 'd']